In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time

In [75]:
option = webdriver.ChromeOptions()
option.add_argument('--headless')
svc = Service('chromedriver.exe')

#search query
q = 'sepeda lipat'
q = q.replace(" ", "%20")
excel_name= q.replace("%20","")
page = 0
maxpage = 5
url_list = []
#define list of url
for i in range(maxpage+1):
    url = 'https://shopee.co.id/search?keyword='+ q +'&page=' + str(page)
    page= page+1
    url_list.append(url)

#get data from urls    
results = []
for url in url_list:
    driver = webdriver.Chrome(service=svc, options=option)
    driver.set_window_size(1300,800)
    driver.get(url)

    #loop for scrolling
    ranges = 500
    for i in range (0,7):
        end = ranges * i
        script = "window.scrollTo(0,"+str(end)+")"
        driver.execute_script(script)
        time.sleep(1)

    time.sleep(3)
    content = driver.page_source
    driver.save_screenshot(excel_name+str(page)+'.png')
    page = page+1
    driver.quit()
    results.append(BeautifulSoup(content,'html.parser'))

# print(data)

In [74]:
i=1
base_url ='https://shopee.co.id'
list_nama, list_pictures, list_price, list_link, list_sold, list_city = [],[],[],[],[],[]
for data in results:
    for item in data.find_all('div', class_="col-xs-2-4 shopee-search-item-result__item"):
        # print('Proses data ke-'+str(i))
        name = item.find('div', class_="ie3A+n bM+7UW Cve6sh").get_text()
        pictures = item.find('img')['src']
        price = item.find('span', class_="ZEgDH9").get_text()
        link = base_url + item.find('a')['href']
        sold = item.find('div', class_="r6HknA uEPGHT")
        if sold != None:
            sold = sold.get_text().split()[0]
        city = item.find('div', class_="zGGwiV").get_text()

        list_nama.append(name)
        list_pictures.append(pictures)
        list_price.append(price)
        list_link.append(link)
        list_sold.append(sold)
        list_city.append(city)

        # print(item)
        i+=1
        # print('-------')
    
    # save to excel
    df = pd.DataFrame({'Nama':list_nama,'Price':list_price,'Sold':list_sold,'City':city, 'Pictures':list_pictures, 'Link':list_link})
    writer = pd.ExcelWriter(excel_name+".xlsx")
    df.to_excel(writer,'Sheet1', index=False)
    writer.save()